# TOI-5807.01 Statistical Validation Walkthrough (TIC 188646744)

This notebook is a **step-by-step validation walkthrough** for a real TESS planet candidate. It is written for astronomers who want to understand *why* each diagnostic is run and what evidence it provides.

It is intended to be the primary end-to-end tutorial for TOI-5807.01, replacing older notebooks over time.

Key principles:

- **Traceable**: each step prints compact, machine-readable summaries.
- **Reproducible**: uses a bundled dataset folder and explicit toggles.
- **Domain-first**: we use astrophysics language (odd/even, secondary eclipse, centroid shifts, dilution, AO constraints) rather than internal check IDs.

You can run this notebook top-to-bottom, or jump directly to the sections you care about.


## Target overview

**TOI-5807.01** (TIC 188646744; HD 196216) is a shallow transit candidate observed by TESS in multiple sectors. The goal here is **statistical validation**: demonstrate that the signal is extremely unlikely to be a false positive and is consistent with an on-target transiting planet.

Evidence categories we will collect:

1. **Light-curve morphology**: planet-like transit shape, no odd/even mismatch, no significant secondary eclipse.
2. **Instrumental/systematics robustness**: consistency across sectors; diagnostics for non-transit phase structure.
3. **On-target localization**: pixel-level centroid/difference-image localization (when per-sector TPFs are available).
4. **Blend constraints**: AO contrast curve reduces probability of unresolved companions.
5. **False Positive Probability (FPP)**: TRICERATOPS(+), incorporating AO constraints.


In [1]:
from __future__ import annotations

import tempfile
from pathlib import Path

import numpy as np

import bittr_tess_vetter.api as btv

# Optional network utilities
from bittr_tess_vetter.api.catalogs import fetch_exofop_toi_table
from bittr_tess_vetter.api.fpp import calculate_fpp

# ----------------------------------------------------------------------------
# Controls (edit these)
# ----------------------------------------------------------------------------
NETWORK = True               # enables catalog lookups and FPP upstream calls
RUN_FPP = True               # set False to skip (can take minutes)
RUN_PIXEL_VETTING = True     # set False to skip (needs lightkurve + network)

# FPP reproducibility knobs (only used when RUN_FPP=True)
FPP_PRESET = "standard"      # "fast" or "standard"
FPP_REPLICATES = 1
FPP_SEED = 42
FPP_TIMEOUT_SECONDS = 1800

TIC_ID = 188646744
TOI_LABEL = "TOI-5807.01"

# Coordinates for catalog queries when NETWORK=True
# (These are only used for catalog-style checks; they do not affect LC-only diagnostics.)
RA_DEG = 304.12005
DEC_DEG = 11.08344


<details>
<summary><b>Expected Output</b></summary>

No output is expected for the imports cell.
</details>


## Run configuration (traceability)

This cell prints the key toggles and library versions. If someone reproduces this notebook, this is the first place to compare environments.


In [2]:
import importlib.metadata

def _pkg_version(name: str) -> str | None:
    try:
        return importlib.metadata.version(name)
    except Exception:
        return None

print(
    {
        "NETWORK": NETWORK,
        "RUN_FPP": RUN_FPP,
        "RUN_PIXEL_VETTING": RUN_PIXEL_VETTING,
        "FPP_PRESET": FPP_PRESET,
        "FPP_REPLICATES": FPP_REPLICATES,
        "FPP_SEED": FPP_SEED,
        "FPP_TIMEOUT_SECONDS": FPP_TIMEOUT_SECONDS,
        "versions": {
            "bittr-tess-vetter": _pkg_version("bittr-tess-vetter"),
            "numpy": _pkg_version("numpy"),
            "astropy": _pkg_version("astropy"),
            "wotan": _pkg_version("wotan"),
            "lightkurve": _pkg_version("lightkurve"),
        },
    }
)


{'NETWORK': True, 'RUN_FPP': True, 'RUN_PIXEL_VETTING': True, 'FPP_PRESET': 'standard', 'FPP_REPLICATES': 1, 'FPP_SEED': 42, 'FPP_TIMEOUT_SECONDS': 1800, 'versions': {'bittr-tess-vetter': '0.1.0', 'numpy': '2.3.5', 'astropy': '7.2.0', 'wotan': '1.10', 'lightkurve': '2.5.1'}}


<details>
<summary><b>Expected Output</b></summary>

```text
{'NETWORK': True, 'RUN_FPP': True, 'RUN_PIXEL_VETTING': True, 'FPP_PRESET': 'standard', 'FPP_REPLICATES': 1, 'FPP_SEED': 42, 'FPP_TIMEOUT_SECONDS': 1800, 'versions': {'bittr-tess-vetter': '0.1.0', 'numpy': '2.3.5', 'astropy': '7.2.0', 'wotan': '1.10', 'lightkurve': '2.5.1'}}
```

</details>


## 1) Load the analysis dataset

We use the bundled tutorial dataset for TOI-5807.01. This makes the workflow runnable even without downloading light curves.

The dataset includes per-sector light curves and a PHARO AO contrast curve.


In [3]:
ds = btv.load_tutorial_target("tic188646744")
ds.summary()

{'schema_version': 1,
 'root': '/Users/collier/projects/apps/bittr-tess-vetter/docs/tutorials/data/tic188646744',
 'sectors_lc': [55, 75, 82, 83],
 'sectors_tpf': [83],
 'artifacts': ['files']}

<details>
<summary><b>Expected Output</b></summary>

```text
{'schema_version': 1,
 'root': '/Users/collier/projects/apps/bittr-tess-vetter/docs/tutorials/data/tic188646744',
 'sectors_lc': [55, 75, 82, 83],
 'sectors_tpf': [83],
 'artifacts': ['files']}
```

</details>


## 2) Candidate ephemeris and stellar context

We pull ExoFOP values when available (NETWORK on). Otherwise we use stable fallback values.

We keep the ephemeris explicit because nearly all diagnostics depend on it.


In [4]:
def _safe_float(x, default=None):
    try:
        return float(x)
    except Exception:
        return default


toi = None
if NETWORK:
    toi_table = fetch_exofop_toi_table()
    matches = toi_table.entries_for_tic(TIC_ID)
    toi = matches[0] if matches else None

# Fallback values (BTJD = BJD - 2457000)
PERIOD_DAYS = _safe_float(toi.get("period_days"), 14.2423724) if toi else 14.2423724
epoch_bjd = _safe_float(toi.get("epoch_bjd"), 2460540.26317) if toi else 2460540.26317
T0_BTJD = (epoch_bjd - 2457000.0) if epoch_bjd and epoch_bjd > 2_450_000 else (epoch_bjd or 3540.26317)
DURATION_HOURS = _safe_float(toi.get("duration_hours"), 4.046) if toi else 4.046
DEPTH_PPM = _safe_float(toi.get("depth_ppm"), 253.0) if toi else 253.0

stellar = btv.StellarParams(
    radius=_safe_float(toi.get("stellar_radius_r_sun"), 1.65) if toi else 1.65,
    mass=_safe_float(toi.get("stellar_mass_m_sun"), 1.47) if toi else 1.47,
    teff=_safe_float(toi.get("stellar_eff_temp_k"), 6816.0) if toi else 6816.0,
    logg=_safe_float(toi.get("stellar_logg"), 4.17) if toi else 4.17,
)

ephem = btv.Ephemeris(period_days=PERIOD_DAYS, t0_btjd=T0_BTJD, duration_hours=DURATION_HOURS)
candidate = btv.Candidate(ephemeris=ephem, depth_ppm=DEPTH_PPM)

print(
    {
        "period_days": ephem.period_days,
        "t0_btjd": ephem.t0_btjd,
        "duration_hours": ephem.duration_hours,
        "depth_ppm": candidate.depth_ppm,
        "stellar_radius_rsun": stellar.radius,
        "stellar_mass_msun": stellar.mass,
    }
)


{'period_days': 14.2423724, 't0_btjd': 3540.2631700001657, 'duration_hours': 4.046, 'depth_ppm': 225.0, 'stellar_radius_rsun': 1.65, 'stellar_mass_msun': 1.47}


<details>
<summary><b>Expected Output</b></summary>

```text
{'period_days': 14.2423724, 't0_btjd': 3540.2631700001657, 'duration_hours': 4.046, 'depth_ppm': 225.0, 'stellar_radius_rsun': 1.65, 'stellar_mass_msun': 1.47}
```

</details>


## 3) Light-curve evidence (planet-like morphology)

We now run the core light-curve vetting checks on a stitched multi-sector light curve:

- **Odd/even test**: an eclipsing binary at half the reported period can show alternating depths.
- **Secondary eclipse search**: a significant eclipse near phase 0.5 is a red flag for an EB.
- **Transit shape / duration / depth stability**: broad consistency checks that the signal behaves like a transit.

These diagnostics should be interpreted as evidence; they are not a planet mass measurement.


In [5]:
wf_default = btv.run_candidate_workflow(
    lc_by_sector=ds.lc_by_sector,
    candidate=candidate,
    stellar=stellar,
    preset="default",
    network=NETWORK,
    ra_deg=RA_DEG,
    dec_deg=DEC_DEG,
    tic_id=TIC_ID,
    run_per_sector=True,
)

print(btv.format_vetting_table(wf_default.bundle))

r01 = wf_default.bundle.get_result("V01")
r02 = wf_default.bundle.get_result("V02")

print(
    {
        "counts": {
            "checks": len(wf_default.bundle.results),
            "ok": wf_default.bundle.n_passed,
            "error": wf_default.bundle.n_failed,
            "skipped": wf_default.bundle.n_unknown,
        },
        "key_metrics": {
            "odd_even_delta_sigma": None if r01 is None else r01.metrics.get("delta_sigma"),
            "secondary_depth_sigma": None if r02 is None else r02.metrics.get("secondary_depth_sigma"),
        },
        "per_sector_summary": wf_default.per_sector.summary_records if wf_default.per_sector else None,
    }
)


Vetting Results
Checks: 15  ok: 12  error: 0  skipped: 3

ID    Name                  Status   Confidence
----  --------------------  -------  ----------
V01   Odd-Even Depth        ok       0.700
V02   Secondary Eclipse     ok       0.850
V03   Duration Consistency  ok       0.850
V04   Depth Stability       ok       0.700
V05   V-Shape               ok       0.935
V06   Nearby EB Search      ok       0.600
V07   ExoFOP TOI Lookup     ok       0.800
V08   Centroid Shift        skipped
V09   Difference Image      skipped
V10   Aperture Dependence   skipped
V11   ModShift              ok       1.000
V11b  ModShiftUniqueness    ok       0.900
V12   SWEET                 ok       1.000
V13   Data Gaps             ok       0.750
V15   Transit Asymmetry     ok       0.750

{'counts': {'checks': 15, 'ok': 12, 'error': 0, 'skipped': 3}, 'key_metrics': {'odd_even_delta_sigma': 1.73, 'secondary_depth_sigma': 1.12}, 'per_sector_summary': [{'sector': 55, 'checks': 15, 'ok': 12, 'error': 0, 'skipp

<details>
<summary><b>Expected Output</b></summary>

```text
Vetting Results
============================================================
Checks: 15  ok: 12  error: 0  skipped: 3

ID    Name                  Status   Confidence
----  --------------------  -------  ----------
V01   Odd-Even Depth        ok       0.700
V02   Secondary Eclipse     ok       0.850
V03   Duration Consistency  ok       0.850
V04   Depth Stability       ok       0.700
V05   V-Shape               ok       0.935
V06   Nearby EB Search      ok       0.600
V07   ExoFOP TOI Lookup     ok       0.800
V08   Centroid Shift        skipped
V09   Difference Image      skipped
V10   Aperture Dependence   skipped
V11   ModShift              ok       1.000
V11b  ModShiftUniqueness    ok       0.900
V12   SWEET                 ok       1.000
V13   Data Gaps             ok       0.750
V15   Transit Asymmetry     ok       0.750

{'counts': {'checks': 15, 'ok': 12, 'error': 0, 'skipped': 3}, 'key_metrics': {'odd_even_delta_sigma': 1.73, 'secondary_depth_sigma': 1.12}, 'per_sector_summary': [{'sector': 55, 'checks': 15, 'ok': 12, 'error': 0, 'skipped': 3}, {'sector': 75, 'checks': 15, 'ok': 12, 'error': 0, 'skipped': 3}, {'sector': 82, 'checks': 15, 'ok': 12, 'error': 0, 'skipped': 3}, {'sector': 83, 'checks': 15, 'ok': 12, 'error': 0, 'skipped': 3}]}
```

</details>


## 4) Robustness and additional diagnostics

Here we run optional metrics-only diagnostics designed to prompt further inspection:

- **Model competition**: does a transit-only model compete well against simple non-transit alternatives?
- **Alias/phase diagnostics**: does the phased light curve show strong non-transit events at other phases?
- **Per-sector ephemeris metrics**: are depth and SNR-like metrics consistent by sector?


In [6]:
wf_ext = btv.run_candidate_workflow(
    lc_by_sector=ds.lc_by_sector,
    candidate=candidate,
    stellar=stellar,
    preset="extended",
    network=NETWORK,
    ra_deg=RA_DEG,
    dec_deg=DEC_DEG,
    tic_id=TIC_ID,
    run_per_sector=True,
)

r16 = wf_ext.bundle.get_result("V16")
r19 = wf_ext.bundle.get_result("V19")

print(
    {
        "V16_model_competition": None if r16 is None else {
            "winner": r16.metrics.get("winner"),
            "label": r16.metrics.get("model_competition_label"),
            "winner_margin_bic": r16.metrics.get("winner_margin_bic"),
            "artifact_risk": r16.metrics.get("artifact_risk"),
        },
        "V19_phase_diagnostics": None if r19 is None else {
            "n_phase_shift_events": r19.metrics.get("n_phase_shift_events"),
            "max_phase_shift_event_sigma": r19.metrics.get("max_phase_shift_event_sigma"),
            "secondary_significance_sigma": r19.metrics.get("secondary_significance_sigma"),
        },
        "sector_ephemeris_metrics": [
            m.to_dict() for m in (wf_ext.per_sector.sector_ephemeris_metrics if wf_ext.per_sector else [])
        ],
    }
)


{'V16_model_competition': {'winner': 'transit_sinusoid', 'label': 'SINUSOID', 'winner_margin_bic': 11.19313001492992, 'artifact_risk': 0.8}, 'V19_phase_diagnostics': {'n_phase_shift_events': 5, 'max_phase_shift_event_sigma': 16.09151334844518, 'secondary_significance_sigma': 0.0}, 'sector_ephemeris_metrics': [{'sector': 55, 'n_total': 18876, 'n_valid': 18876, 'time_start_btjd': 2797.1034851753184, 'time_end_btjd': 2824.2660853937973, 'duration_days': 27.16260021847893, 'cadence_seconds': 120.00026772147976, 'n_in_transit': 243, 'n_out_of_transit': 18147, 'depth_hat_ppm': 169.3996568745224, 'depth_sigma_ppm': 13.385897370705903, 'score': 12.655084092101413, 'flux_mad_ppm': 183.62453236769883}, {'sector': 75, 'n_total': 19470, 'n_valid': 19470, 'time_start_btjd': 3339.7823703707395, 'time_end_btjd': 3367.4793730420656, 'duration_days': 27.697002671326118, 'cadence_seconds': 119.99909746373305, 'n_in_transit': 242, 'n_out_of_transit': 18742, 'depth_hat_ppm': 308.6055415614641, 'depth_sigm

<details>
<summary><b>Expected Output</b></summary>

```text
{'V16_model_competition': {'winner': 'transit_sinusoid', 'label': 'SINUSOID', 'winner_margin_bic': 11.19313001492992, 'artifact_risk': 0.8}, 'V19_phase_diagnostics': {'n_phase_shift_events': 5, 'max_phase_shift_event_sigma': 16.09151334844518, 'secondary_significance_sigma': 0.0}, 'sector_ephemeris_metrics': [{'sector': 55, 'n_total': 18876, 'n_valid': 18876, 'time_start_btjd': 2797.1034851753184, 'time_end_btjd': 2824.2660853937973, 'duration_days': 27.16260021847893, 'cadence_seconds': 120.00026772147976, 'n_in_transit': 243, 'n_out_of_transit': 18147, 'depth_hat_ppm': 169.3996568745224, 'depth_sigma_ppm': 13.385897370705903, 'score': 12.655084092101413, 'flux_mad_ppm': 183.62453236769883}, {'sector': 75, 'n_total': 19470, 'n_valid': 19470, 'time_start_btjd': 3339.7823703707395, 'time_end_btjd': 3367.4793730420656, 'duration_days': 27.697002671326118, 'cadence_seconds': 119.99909746373305, 'n_in_transit': 242, 'n_out_of_transit': 18742, 'depth_hat_ppm': 308.6055415614641, 'depth_sigma_ppm': 13.61163127930717, 'score': 22.672193745845583, 'flux_mad_ppm': 174.83799128414913}, {'sector': 82, 'n_total': 18133, 'n_valid': 18133, 'time_start_btjd': 3533.395836686563, 'time_end_btjd': 3559.2485963848135, 'duration_days': 25.85275969825034, 'cadence_seconds': 119.99992408964317, 'n_in_transit': 243, 'n_out_of_transit': 17404, 'depth_hat_ppm': 225.20933736015078, 'depth_sigma_ppm': 13.388749341720294, 'score': 16.820789725174887, 'flux_mad_ppm': 169.37067747402335}, {'sector': 83, 'n_total': 17326, 'n_valid': 17326, 'time_start_btjd': 3559.458315945771, 'time_end_btjd': 3584.377208025556, 'duration_days': 24.918892079784655, 'cadence_seconds': 119.99736897123512, 'n_in_transit': 244, 'n_out_of_transit': 16598, 'depth_hat_ppm': 309.3230302814298, 'depth_sigma_ppm': 13.435611843556007, 'score': 23.022623300165332, 'flux_mad_ppm': 163.72480254656097}]}
```

</details>


## 5) On-target localization (pixel-level)

A major false-positive mode for TESS is a **blended eclipsing binary**: the transit-like signal originates from a nearby source within the photometric aperture.

Pixel-level vetting uses per-sector Target Pixel Files (TPFs) to check whether the in-transit centroid / difference image points to the target location.

The bundled dataset contains only one sector TPF stamp. To do this properly across sectors, we download per-sector TPFs.


In [7]:
if not RUN_PIXEL_VETTING:
    print("Skipping pixel-level vetting (set RUN_PIXEL_VETTING=True).")
else:
    if not NETWORK:
        raise ValueError("Set NETWORK=True to download per-sector TPFs")
    import lightkurve as lk

    tpf_by_sector: dict[int, btv.TPFStamp] = {}
    for sector in sorted(ds.lc_by_sector.keys()):
        search = lk.search_targetpixelfile(f"TIC {TIC_ID}", sector=int(sector), exptime=120)
        if len(search) == 0:
            print(f"No TPF for sector {sector}")
            continue
        tpf = search.download()
        if tpf is None:
            print(f"Download failed for sector {sector}")
            continue
        tpf_by_sector[int(sector)] = btv.TPFStamp(
            time=np.asarray(tpf.time.value, dtype=np.float64),
            flux=np.asarray(tpf.flux.value, dtype=np.float64),
            flux_err=np.asarray(tpf.flux_err.value, dtype=np.float64),
            wcs=tpf.wcs,
            aperture_mask=np.asarray(tpf.pipeline_mask, dtype=bool),
            quality=np.asarray(tpf.quality, dtype=np.int32),
        )

    per = btv.per_sector_vet(
        ds.lc_by_sector,
        candidate,
        stellar=stellar,
        tpf_by_sector=tpf_by_sector,
        preset="extended",
        network=False,
    )

    for sector, bundle in sorted(per.bundles_by_sector.items()):
        r_centroid = bundle.get_result("V08")
        r_diffimg = bundle.get_result("V09")
        r_apdep = bundle.get_result("V10")
        print(
            {
                "sector": int(sector),
                "centroid_shift_status": None if r_centroid is None else r_centroid.status,
                "difference_image_status": None if r_diffimg is None else r_diffimg.status,
                "aperture_dependence_status": None if r_apdep is None else r_apdep.status,
            }
        )


/Users/collier/projects/apps/bittr-tess-vetter/.venv/lib/python3.12/site-packages/lightkurve/prf/__init__.py:7: UserWarning: Warning: the tpfmodel submodule is not available without oktopus installed, which requires a current version of autograd. See #1452 for details.
  warnings.warn(


{'sector': 55, 'centroid_shift_status': 'ok', 'difference_image_status': 'ok', 'aperture_dependence_status': 'ok'}
{'sector': 75, 'centroid_shift_status': 'ok', 'difference_image_status': 'ok', 'aperture_dependence_status': 'ok'}
{'sector': 82, 'centroid_shift_status': 'ok', 'difference_image_status': 'ok', 'aperture_dependence_status': 'ok'}
{'sector': 83, 'centroid_shift_status': 'ok', 'difference_image_status': 'ok', 'aperture_dependence_status': 'ok'}


<details>
<summary><b>Expected Output</b></summary>

```text
/Users/collier/projects/apps/bittr-tess-vetter/.venv/lib/python3.12/site-packages/lightkurve/prf/__init__.py:7: UserWarning: Warning: the tpfmodel submodule is not available without oktopus installed, which requires a current version of autograd. See #1452 for details.
  warnings.warn(
{'sector': 55, 'centroid_shift_status': 'ok', 'difference_image_status': 'ok', 'aperture_dependence_status': 'ok'}
{'sector': 75, 'centroid_shift_status': 'ok', 'difference_image_status': 'ok', 'aperture_dependence_status': 'ok'}
{'sector': 82, 'centroid_shift_status': 'ok', 'difference_image_status': 'ok', 'aperture_dependence_status': 'ok'}
{'sector': 83, 'centroid_shift_status': 'ok', 'difference_image_status': 'ok', 'aperture_dependence_status': 'ok'}
```

</details>


## 6) AO-assisted False Positive Probability (FPP)

Statistical validation typically uses **FPP < 1%** as a community convention. Here we compute TRICERATOPS(+), incorporating the PHARO AO contrast curve to rule out unresolved companions.

This is the most time-consuming step and involves upstream models (Gaia + TRILEGAL). We make it as reproducible as practical by setting an explicit seed.


In [8]:
if not RUN_FPP:
    print("Skipping FPP (set RUN_FPP=True).")
else:
    if not NETWORK:
        raise ValueError("Set NETWORK=True for FPP")

    cc = btv.load_contrast_curve_exofop_tbl(ds.root / "PHARO_Kcont_plot.tbl", filter="Kcont")
    cache_dir = Path(tempfile.mkdtemp(prefix="btv_tutorial_fpp_"))
    cache = btv.hydrate_cache_from_dataset(
        dataset=ds,
        tic_id=TIC_ID,
        flux_type="pdcsap",
        cache_dir=cache_dir,
        cadence_seconds=120.0,
    )
    print(f"Cache directory: {cache_dir}")

    fpp = calculate_fpp(
        cache=cache,
        tic_id=TIC_ID,
        period=ephem.period_days,
        t0=ephem.t0_btjd,
        depth_ppm=float(candidate.depth_ppm or DEPTH_PPM),
        duration_hours=ephem.duration_hours,
        sectors=sorted(ds.lc_by_sector.keys()),
        stellar_radius=stellar.radius,
        stellar_mass=stellar.mass,
        preset=FPP_PRESET,
        timeout_seconds=FPP_TIMEOUT_SECONDS,
        replicates=FPP_REPLICATES,
        seed=FPP_SEED,
        contrast_curve=cc,
    )

    print(fpp)


Cache directory: /var/folders/7_/ndc1rbkx4xs20nph76f_96rc0000gn/T/btv_tutorial_fpp_2nogynld


/Users/collier/projects/apps/bittr-tess-vetter/.venv/lib/python3.12/site-packages/pytransit/stars/btsettl.py:37: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename
/Users/collier/projects/apps/bittr-tess-vetter/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No apertures provided, assuming 5x5 centered on target.


griz mags: nan nan nan nan


Z, Teff, logg: 0.0 6750 4.0


...

griz mags: nan nan nan nan


Z, Teff, logg: 0.0 5750 4.5


griz mags: 17.8105 16.1312 15.4018 14.8373


Z, Teff, logg: 0.0 6750 4.0


{'error': 'All 1 replicate(s) returned degenerate/invalid posteriors', 'error_type': 'degenerate_posterior', 'stage': 'replicate_aggregation', 'tic_id': 188646744, 'tmag': None, 'sectors_used': [55, 75, 82, 83], 'runtime_seconds': 488.4, 'replicates': 1, 'n_success': 0, 'n_fail': 1, 'base_seed': 42, 'degenerate_reasons': ['fpp_not_finite,posterior_sum_not_finite,posterior_prob_nan_count=30'], 'replicate_errors': []}


<details>
<summary><b>Expected Output</b></summary>

```text
Cache directory: /var/folders/7_/ndc1rbkx4xs20nph76f_96rc0000gn/T/btv_tutorial_fpp_2nogynld
/Users/collier/projects/apps/bittr-tess-vetter/.venv/lib/python3.12/site-packages/pytransit/stars/btsettl.py:37: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename
/Users/collier/projects/apps/bittr-tess-vetter/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No apertures provided, assuming 5x5 centered on target.
WARNING: 188646747 is missing stellar properties. If a mass (in M_Sun), radius (in R_Sun), and/or Teff (in K) are not added to the .stars dataframe, Solar values will be assumed.
WARNING: 188646750 is missing stellar properties. If a mass (in M_Sun), radius (in R_Sun), and/or Teff (in K) are not added to the .stars dataframe, Solar values will be assumed.
WARNING: 188519181 is missing stellar properties. If a mass (in M_Sun), radius (in R_Sun), and/or Teff (in K) are not added to the .stars dataframe, Solar values will be assumed.
WARNING: 188646760 is missing stellar properties. If a mass (in M_Sun), radius (in R_Sun), and/or Teff (in K) are not added to the .stars dataframe, Solar values will be assumed.
griz mags: nan nan nan nan
Z, Teff, logg: 0.0 6750 4.0
/Users/collier/projects/apps/bittr-tess-vetter/src/bittr_tess_vetter/ext/triceratops_plus_vendor/triceratops/funcs.py:187: RuntimeWarning: overflow encountered in power
  return 10**flux_spline(Masses)
/Users/collier/projects/apps/bittr-tess-vetter/src/bittr_tess_vetter/ext/triceratops_plus_vendor/triceratops/funcs.py:193: RuntimeWarning: overflow encountered in power
  return 10**flux_spline_K(Masses)
griz mags: nan nan nan nan
Z, Teff, logg: 0.0 5750 4.5
griz mags: nan nan nan nan
Z, Teff, logg: 0.0 5750 4.5
griz mags: nan nan nan nan
Z, Teff, logg: 0.0 5750 4.5
griz mags: nan nan nan nan
Z, Teff, logg: 0.0 5750 4.5
griz mags: 17.8105 16.1312 15.4018 14.8373
Z, Teff, logg: 0.0 6750 4.0
{'error': 'All 1 replicate(s) returned degenerate/invalid posteriors', 'error_type': 'degenerate_posterior', 'stage': 'replicate_aggregation', 'tic_id': 188646744, 'tmag': None, 'sectors_used': [55, 75, 82, 83], 'runtime_seconds': 488.4, 'replicates': 1, 'n_success': 0, 'n_fail': 1, 'base_seed': 42, 'degenerate_reasons': ['fpp_not_finite,posterior_sum_not_finite,posterior_prob_nan_count=30'], 'replicate_errors': []}
```

</details>


## 7) Export a shareable report

We export a markdown report (for human review) and a JSON bundle (for programmatic auditing).


In [9]:
out_dir = Path(tempfile.mkdtemp(prefix="btv_toi5807_trace_"))

btv.export_bundle(wf_ext.bundle, format="md", path=out_dir / "report.md", title=f"{TOI_LABEL} ({TIC_ID})")
btv.export_bundle(wf_ext.bundle, format="json", path=out_dir / "bundle.json")

print({"out_dir": str(out_dir), "files": sorted(p.name for p in out_dir.iterdir())})


{'out_dir': '/var/folders/7_/ndc1rbkx4xs20nph76f_96rc0000gn/T/btv_toi5807_trace_98wewqmz', 'files': ['bundle.json', 'report.md']}


<details>
<summary><b>Expected Output</b></summary>

```text
{'out_dir': '/var/folders/7_/ndc1rbkx4xs20nph76f_96rc0000gn/T/btv_toi5807_trace_98wewqmz', 'files': ['bundle.json', 'report.md']}
```

</details>


## Appendix: mapping to implementation checks

Internally, the library runs a set of named checks. You do not need to memorize IDs to use this tutorial, but if you want an audit trail:

- Odd/even depth consistency → `V01`
- Secondary eclipse search → `V02`
- Pixel centroid shift → `V08`
- Difference image localization → `V09`
- Aperture dependence → `V10`
- Model competition (transit vs non-transit alternatives) → `V16`
- Alias/phase diagnostics (phase-shift events) → `V19`

The markdown report exported above includes the full per-check provenance.
